# Первая генерация рекомендаций

## Вывод предпочтений пользователя

In [41]:
import pandas as pd

# Загружаем DataFrame из файла Parquet
df = pd.read_parquet('users/user0_tags.parquet')

# Выводим первые 5 строк
print(df.head(100))

                             tag  percent  likes
0                    Мультфильмы      0.0      0
1                    Развлечения      0.0      0
2                          Хобби      0.0      0
3                       Животные      0.0      0
4                         Разное      0.0      0
5                      Лайфстайл      0.0      0
6                   Телепередачи      0.0      0
7                          Спорт      0.0      0
8                   Недвижимость      0.0      0
9                        Сериалы      0.0      0
10                         Детям      0.0      0
11                        Музыка      0.0      0
12                          Юмор      0.0      0
13                     Авто-мото      0.0      0
14                  Сад и огород      0.0      0
15                        Фильмы      0.0      0
16                     Видеоигры      0.0      0
17                         Наука      0.0      0
18                   Путешествия      0.0      0
19         Технологи

## Стандартная генерация исходя из предпочтений пользователя (т.к. пользователь холодный, то и выводится будут с платформы популярные категории)

In [42]:
import pandas as pd
import joblib
import random
import time

# Загрузка модели
model = joblib.load('random_forest_model.joblib')

# Загрузка данных из файлов
filtred_df = pd.read_parquet('tags-filtered.parquet')
logs_df = pd.read_parquet('logsV02.parquet')
tags_df = pd.read_parquet('users/user0_tags.parquet')
regions_df = pd.read_parquet('users/user0_regions.parquet')
city_df = pd.read_parquet('users/user0_city.parquet')
additional_df = pd.read_parquet('parquet-filtered/filtred.parquet')

# Объединение данных по video_id
merged_df = pd.merge(filtred_df, logs_df, on='video_id', how='inner')

# Ограничиваемся первыми 1000 записями
merged_df = merged_df.head(1000)

# Признаки
features = ['category_id', 'v_likes', 'v_dislikes',
            'v_frac_avg_watchtime_1_day_duration',
            'v_frac_avg_watchtime_7_day_duration',
            'v_frac_avg_watchtime_30_day_duration']

# Преобразование категориальных переменных для новых данных
X_new = pd.get_dummies(merged_df[features], drop_first=True)

# Обработка данных: замена бесконечных значений и NaN
X_new.replace([float('inf'), float('-inf')], pd.NA, inplace=True)
X_new.dropna(inplace=True)

# Сохранение индексов оставшихся строк
valid_indices = X_new.index

# Обеспечение соответствия признаков модели
X_new = X_new.reindex(columns=model.feature_names_in_, fill_value=0)

# Выполнение предсказаний
predictions = model.predict(X_new)

# Создание нового столбца с предсказанными значениями только для действительных индексов
merged_df.loc[valid_indices, 'predicted_popularity'] = predictions

# Удаление дубликатов по video_id
unique_videos = merged_df.drop_duplicates(subset='video_id')

# Сортировка по предсказанной популярности
unique_videos = unique_videos.sort_values(by='predicted_popularity', ascending=False)

# Поиск максимальных 3 значений percent с тегом
top_tags = tags_df.nlargest(3, 'percent')

# Список для хранения финальных видео
final_video_ids = []

# Выбор случайного числа для изменения выборки
random.seed(time.time())

# Поиск двух популярных видео для каждого из топовых тегов
for tag in top_tags['tag']:
    # Находим все видео с данным тегом
    videos_with_tag = unique_videos[unique_videos['category_id'].str.contains(tag, na=False)]
    
    if not videos_with_tag.empty:
        # Выбор до 2 видео с этим тегом
        selected_videos = videos_with_tag.sample(n=min(2, len(videos_with_tag)), random_state=random.randint(0, 100))
        final_video_ids.extend(selected_videos['video_id'].tolist())

# Если всё ещё не достигли 10 видео, добавляем случайные видео из других категорий
remaining_slots = 10 - len(final_video_ids)

if remaining_slots > 0:
    popular_others = unique_videos[~unique_videos['video_id'].isin(final_video_ids)].sample(
        n=min(remaining_slots, len(unique_videos[~unique_videos['video_id'].isin(final_video_ids)])),
              random_state=random.randint(0, 100))
        
    final_video_ids.extend(popular_others['video_id'].tolist())

# Удаляем дубликаты, если они есть
final_video_ids = list(set(final_video_ids))

# Получаем финальные 10 видео
final_video_ids = final_video_ids[:10]

# Проверка на наличие файла security.parquet и его содержимого
try:
    security_df = pd.read_parquet('security.parquet')
except ValueError:
    # Если файл не существует или пуст, то создаем пустая DataFrame
    security_df = pd.DataFrame(columns=['video_id'])

if security_df.empty:
    # Если файл пуст, просто используем финальные video_id
    existing_video_ids = set()  # Пустое множество, так как ничего нет в security.parquet
else:
    existing_video_ids = set(security_df['video_id'])

# Генерация новых video_id в случае совпадений
while any(video_id in existing_video_ids for video_id in final_video_ids):
    # Ищем альтернативные video_id, пока все не будут уникальными
    additional_videos = unique_videos[~unique_videos['video_id'].isin(final_video_ids)].sample(
    n=len(final_video_ids), random_state=random.randint(0, 100))
    
    final_video_ids = additional_videos['video_id'].tolist()
    final_video_ids = list(set(final_video_ids))[:10]  # Убедимся, что оставим только 10

# Получаем данные о финальных видео
final_videos = unique_videos[unique_videos['video_id'].isin(final_video_ids)]

# Объединяем с дополнительным DataFrame для получения title и v_pub_datetime
final_videos_info = pd.merge(final_videos, additional_df[['video_id', 'title', 'v_pub_datetime']],
                              on='video_id', how='left')

# Сохранение итоговых video_id в файл security.parquet
final_video_ids_df = pd.DataFrame(final_video_ids, columns=['video_id'])

# Добавляем новые video_id в security.parquet, если они уникальны
new_ids_to_add = final_video_ids_df[~final_video_ids_df['video_id'].isin(existing_video_ids)]

if not new_ids_to_add.empty:
    # Объединяем с существующими данными
    updated_security_df = pd.concat([security_df, new_ids_to_add], ignore_index=True)
    updated_security_df.to_parquet('security.parquet', index=False)

# Вывод финальных видео
print(final_videos_info[['video_id', 'title', 'v_pub_datetime', 'category_id']])
# Сохранение данных в файл final_videos_info.parquet
final_videos_info[['video_id', 'title', 'v_pub_datetime', 'category_id']].to_parquet('final_videos_info.parquet', index=False)

print("Данные успешно сохранены в файл final_videos_info.parquet.")

                               video_id  \
0  e0eedb0a-2468-4193-8b76-2c41fd9af529   
1  15761c10-583a-4e3a-84db-f7f48fa94782   
2  dde49c8e-db30-427f-bef5-3f94701ede0b   
3  82bf16f6-9764-4582-9607-fe98d7c6ceba   
4  f0086c15-3718-411b-ad40-bc1fb67f67cb   
5  58aac622-479c-4011-ae6d-1001b52a9a50   
6  cf26723f-121c-448f-ae64-1b2430c4a854   
7  8dfb4533-dd5c-432a-aefe-fa6e1c18f3bc   
8  9ba1f34d-935a-4c43-a5ce-ab78cdd9f407   
9  11449640-6224-4f1c-8cef-38270b1a248b   

                                               title  \
0              Отчаянные Домохозяйки 1 сезон 3 серия   
1  Сериал Презумпция невиновности – 1 сезон 1 сер...   
2  Магическая битва 2 сезон 15 серия (аниме-сериа...   
3  Gravity Falls - (2 сезон) 4 серия "Носочная оп...   
4                      10 глупых вопросов ШОУРАННЕРУ   
5  Новое Простоквашино – 11 серия – Вишнёвая Феер...   
6  Домой - любой ценой. Мужское / Женское. Самые ...   
7                             Самые необычные КНИГИ!   
8  Возрастающей или уб

## Пользователь ставит лайк, дизлайк на видео

## Сбор и сохранение результатов первой выдачи

In [43]:
import pandas as pd
import json

# Загрузка данных из файла final_videos_info.parquet
final_videos_info = pd.read_parquet('final_videos_info.parquet')

# Считывание нужных столбцов
video_data = final_videos_info[['video_id', 'title', 'category_id']]

# Инициализация списка для итоговых значений
aggregated = []

# Запрос значений лайков и дизлайков
for index, row in video_data.iterrows():
    video_id = row['video_id']
    title = row['title']
    category_id = row['category_id']
    
    total_likes = int(input(f"Значение лайка для видео (0 или 1) [{title}] категории [{category_id}]: "))
    total_dislikes = int(input(f"Значение дизлайка для видео (0 или 1) [{title}] категории [{category_id}]: "))
    
    aggregated.append({
        'uid': video_id,
        'total_likes': total_likes,
        'total_dislikes': total_dislikes
    })

# Сохранение итоговых значений в файл JSON
with open('aggregated_data.json', 'w') as json_file:
    json.dump(aggregated, json_file, indent=4)

# Вывод содержимого файла JSON
with open('aggregated_data.json', 'r') as json_file:
    content = json_file.read()
    #print("Содержимое файла aggregated_data.json:")
    #print(content)

Значение лайка для видео (0 или 1) [Отчаянные Домохозяйки 1 сезон 3 серия] категории [Сериалы]:  1
Значение дизлайка для видео (0 или 1) [Отчаянные Домохозяйки 1 сезон 3 серия] категории [Сериалы]:  0
Значение лайка для видео (0 или 1) [Сериал Презумпция невиновности – 1 сезон 1 серия  / Presumed Innocent] категории [Сериалы]:  1
Значение дизлайка для видео (0 или 1) [Сериал Презумпция невиновности – 1 сезон 1 серия  / Presumed Innocent] категории [Сериалы]:  0
Значение лайка для видео (0 или 1) [Магическая битва 2 сезон 15 серия (аниме-сериал, 2023)] категории [Разное]:  1
Значение дизлайка для видео (0 или 1) [Магическая битва 2 сезон 15 серия (аниме-сериал, 2023)] категории [Разное]:  0
Значение лайка для видео (0 или 1) [Gravity Falls - (2 сезон) 4 серия "Носочная опера"] категории [Мультфильмы]:  0
Значение дизлайка для видео (0 или 1) [Gravity Falls - (2 сезон) 4 серия "Носочная опера"] категории [Мультфильмы]:  0
Значение лайка для видео (0 или 1) [10 глупых вопросов ШОУРАННЕРУ]

## Подсчёт интересов пользователя

In [44]:
import pandas as pd
import json

# Загрузка данных из файлов
tags_df = pd.read_parquet('users/user0_tags.parquet')
categories_df = pd.read_parquet('tags-filtered.parquet')

# Загрузка данных из файла aggregated_data.json
with open('aggregated_data.json', 'r') as json_file:
    aggregated = json.load(json_file)

# Инициализация словаря для накопления лайков и дизлайков по категориям
category_likes = {}

# Суммирование лайков и дизлайков для каждой категории
for item in aggregated:
    video_id = item['uid']
    
    # Получение category_id по video_id
    category_info = categories_df[categories_df['video_id'] == video_id]
    
    if not category_info.empty:
        category_id = category_info['category_id'].values[0]
        
        if category_id not in category_likes:
            category_likes[category_id] = {'likes': 0, 'dislikes': 0}
        
        category_likes[category_id]['likes'] += item['total_likes']
        category_likes[category_id]['dislikes'] += item['total_dislikes']

# Обновление тегов на основе накопленных значений
for category_id, counts in category_likes.items():
    tag_index = tags_df[tags_df['tag'] == category_id].index
    
    if not tag_index.empty:
        tag_row = tag_index[0]
        
        # Обновляем likes в tags_df
        tags_df.at[tag_row, 'likes'] += counts['likes']
        # Учитываем дизлайки
        tags_df.at[tag_row, 'likes'] = max(0, tags_df.at[tag_row, 'likes'] - counts['dislikes'])

# Функция для расчета процентов
def calculate_percentages(df):
    total_likes = df['likes'].sum()
    if total_likes > 0:
        df['percent'] = (df['likes'] / total_likes * 100).fillna(0)
    else:
        df['percent'] = 0
    return df

# Расчет лайков и процентов для тегов
tags_df = calculate_percentages(tags_df)

# Сохранение обновленного DataFrame обратно в файл
tags_df.to_parquet('users/user0_tags.parquet', index=False)

print("Лайки и процентное соотношение сохранены в файл users/user0_tags.parquet.")

Лайки и процентное соотношение сохранены в файл users/user0_tags.parquet.


In [45]:
import pandas as pd

# Загружаем DataFrame из файла Parquet
df = pd.read_parquet('users/user0_tags.parquet')

# Выводим первые 5 строк
print(df.head(100))

                             tag    percent  likes
0                    Мультфильмы   0.000000      0
1                    Развлечения   0.000000      0
2                          Хобби   0.000000      0
3                       Животные   0.000000      0
4                         Разное  33.333333      1
5                      Лайфстайл   0.000000      0
6                   Телепередачи   0.000000      0
7                          Спорт   0.000000      0
8                   Недвижимость   0.000000      0
9                        Сериалы  66.666667      2
10                         Детям   0.000000      0
11                        Музыка   0.000000      0
12                          Юмор   0.000000      0
13                     Авто-мото   0.000000      0
14                  Сад и огород   0.000000      0
15                        Фильмы   0.000000      0
16                     Видеоигры   0.000000      0
17                         Наука   0.000000      0
18                   Путешестви

# Вывод рекомендация исходя из интересов пользователя

In [46]:
import pandas as pd
import joblib
import random
import time

# Загрузка модели
model = joblib.load('random_forest_model.joblib')

# Загрузка данных из файлов
filtred_df = pd.read_parquet('tags-filtered.parquet')
logs_df = pd.read_parquet('logsV02.parquet')
tags_df = pd.read_parquet('users/user0_tags.parquet')
regions_df = pd.read_parquet('users/user0_regions.parquet')
city_df = pd.read_parquet('users/user0_city.parquet')
additional_df = pd.read_parquet('parquet-filtered/filtred.parquet')

# Объединение данных по video_id
merged_df = pd.merge(filtred_df, logs_df, on='video_id', how='inner')

# Ограничиваемся первыми 1000 записями
merged_df = merged_df.head(1000)

# Признаки
features = ['category_id', 'v_likes', 'v_dislikes',
            'v_frac_avg_watchtime_1_day_duration',
            'v_frac_avg_watchtime_7_day_duration',
            'v_frac_avg_watchtime_30_day_duration']

# Преобразование категориальных переменных для новых данных
X_new = pd.get_dummies(merged_df[features], drop_first=True)

# Обработка данных: замена бесконечных значений и NaN
X_new.replace([float('inf'), float('-inf')], pd.NA, inplace=True)
X_new.dropna(inplace=True)

# Сохранение индексов оставшихся строк
valid_indices = X_new.index

# Обеспечение соответствия признаков модели
X_new = X_new.reindex(columns=model.feature_names_in_, fill_value=0)

# Выполнение предсказаний
predictions = model.predict(X_new)

# Создание нового столбца с предсказанными значениями только для действительных индексов
merged_df.loc[valid_indices, 'predicted_popularity'] = predictions

# Удаление дубликатов по video_id
unique_videos = merged_df.drop_duplicates(subset='video_id')

# Сортировка по предсказанной популярности
unique_videos = unique_videos.sort_values(by='predicted_popularity', ascending=False)

# Поиск максимальных 3 значений percent с тегом
top_tags = tags_df.nlargest(3, 'percent')

# Список для хранения финальных видео
final_video_ids = []

# Выбор случайного числа для изменения выборки
random.seed(time.time())

# Поиск двух популярных видео для каждого из топовых тегов
for tag in top_tags['tag']:
    # Находим все видео с данным тегом
    videos_with_tag = unique_videos[unique_videos['category_id'].str.contains(tag, na=False)]
    
    if not videos_with_tag.empty:
        # Выбор до 2 видео с этим тегом
        selected_videos = videos_with_tag.sample(n=min(2, len(videos_with_tag)), random_state=random.randint(0, 100))
        final_video_ids.extend(selected_videos['video_id'].tolist())

# Если всё ещё не достигли 10 видео, добавляем случайные видео из других категорий
remaining_slots = 10 - len(final_video_ids)

if remaining_slots > 0:
    popular_others = unique_videos[~unique_videos['video_id'].isin(final_video_ids)].sample(
        n=min(remaining_slots, len(unique_videos[~unique_videos['video_id'].isin(final_video_ids)])),
              random_state=random.randint(0, 100))
        
    final_video_ids.extend(popular_others['video_id'].tolist())

# Удаляем дубликаты, если они есть
final_video_ids = list(set(final_video_ids))

# Получаем финальные 10 видео
final_video_ids = final_video_ids[:10]

# Проверка на наличие файла security.parquet и его содержимого
try:
    security_df = pd.read_parquet('security.parquet')
except ValueError:
    # Если файл не существует или пуст, то создаем пустая DataFrame
    security_df = pd.DataFrame(columns=['video_id'])

if security_df.empty:
    # Если файл пуст, просто используем финальные video_id
    existing_video_ids = set()  # Пустое множество, так как ничего нет в security.parquet
else:
    existing_video_ids = set(security_df['video_id'])

# Генерация новых video_id в случае совпадений
while any(video_id in existing_video_ids for video_id in final_video_ids):
    # Ищем альтернативные video_id, пока все не будут уникальными
    additional_videos = unique_videos[~unique_videos['video_id'].isin(final_video_ids)].sample(
    n=len(final_video_ids), random_state=random.randint(0, 100))
    
    final_video_ids = additional_videos['video_id'].tolist()
    final_video_ids = list(set(final_video_ids))[:10]  # Убедимся, что оставим только 10

# Получаем данные о финальных видео
final_videos = unique_videos[unique_videos['video_id'].isin(final_video_ids)]

# Объединяем с дополнительным DataFrame для получения title и v_pub_datetime
final_videos_info = pd.merge(final_videos, additional_df[['video_id', 'title', 'v_pub_datetime']],
                              on='video_id', how='left')

# Сохранение итоговых video_id в файл security.parquet
final_video_ids_df = pd.DataFrame(final_video_ids, columns=['video_id'])

# Добавляем новые video_id в security.parquet, если они уникальны
new_ids_to_add = final_video_ids_df[~final_video_ids_df['video_id'].isin(existing_video_ids)]

if not new_ids_to_add.empty:
    # Объединяем с существующими данными
    updated_security_df = pd.concat([security_df, new_ids_to_add], ignore_index=True)
    updated_security_df.to_parquet('security.parquet', index=False)

# Вывод финальных видео
print(final_videos_info[['video_id', 'title', 'v_pub_datetime', 'category_id']])
# Сохранение данных в файл final_videos_info.parquet
final_videos_info[['video_id', 'title', 'v_pub_datetime', 'category_id']].to_parquet('final_videos_info.parquet', index=False)

print("Данные успешно сохранены в файл final_videos_info.parquet.")

                               video_id  \
0  3c43a697-fa07-48ec-91ac-80f5706252b3   
1  7644ebc0-65f9-4113-921e-98629a5ff1bc   
2  12c0dcb0-eed5-4c10-b734-df6a90de6112   
3  f603a751-9223-4157-b248-9250b4379cce   
4  bcd43ee0-425a-4d3a-aa00-09a80bb120f7   
5  4126d9a8-955e-454f-9997-6ba52c698906   
6  53d7fd98-ba32-4913-86a9-11ee1f01948f   
7  e405ebb5-9091-4b46-9552-2810c274a58d   
8  1774ca13-4edc-4816-b999-e7c3d279e91d   
9  478cceab-4fff-4b16-b65a-caaf91bd23c6   

                                               title  \
0  Хороший доктор – 2 сезон 4 серия «Крепкий ореш...   
1  Хороший доктор – 4 сезон 20 серия «Начнём» / T...   
2  Обоюдное согласие 1 Сезон _ 3 Серия _ Сюжет и ...   
3      Гравити Фолз (2 сезон 18 серия) – Апокалипсис   
4  Военнослужащий ищет женщину в соку. Давай поже...   
5              Король: Вечный монарх 1 сезон 1 серия   
6  Минифорс. Сила динозавров, 51 серия. Лорд полу...   
7                              С праздником 8 марта!   
8  СУПЕР СРЕДСТВО ,👍 к